### Import Library

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split

### Read the dataset of csv file

In [2]:
df_train = pd.read_csv('BankChurners_preprocess.csv')
df_train.head()

,CustomerId,France,Germany,Spain,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,CreditLevel
0,15762418,0.0,0.0,1.0,0.3,0.484985,0.000000,1,0,0.643290,1,7
1,15749905,0.0,0.0,1.0,0.6,0.000000,0.000000,1,0,0.251062,1,6
2,15600911,1.0,0.0,0.0,0.2,0.728934,0.000000,1,0,0.015250,0,6
3,15572762,0.0,1.0,0.0,0.2,0.407651,0.333333,1,0,0.449146,0,1
4,15627848,1.0,0.0,0.0,0.7,0.435819,0.333333,1,0,0.513377,0,6


#### Split the data into train and test set

In [3]:
train, test = train_test_split(df_train, test_size=0.2, random_state = 1)

train_y = pd.DataFrame(train['CreditLevel'])
train_X = train.drop(columns=['CreditLevel'])
train_X = train_X.drop(columns=['CustomerId'])

test_y = pd.DataFrame(test['CreditLevel'])
test_X = test.drop(columns=['CreditLevel'])
test_X = test_X.drop(columns=['CustomerId'])

In [4]:
train_set = torch.utils.data.TensorDataset(torch.Tensor(np.array(train_X)), torch.squeeze(torch.Tensor(np.array(train_y))))
test_set = torch.utils.data.TensorDataset(torch.Tensor(np.array(test_X)), torch.squeeze(torch.Tensor(np.array(test_y))))
            

train_loader = torch.utils.data.DataLoader(dataset=train_set, 
                                           batch_size=64, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_set, 
                                          batch_size=64, 
                                          shuffle=False)

### Network Architecture

In [5]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()

        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 10)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x),negative_slope=0.01)
        x = F.leaky_relu(self.fc2(x),negative_slope=0.01)
        x = F.softmax(self.fc3(x))
        return x

#### Loss function and learning rate

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  

### Train

In [7]:
total_step = len(train_loader)
num_epochs = 50

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        inputs = inputs.to(device)
        labels = torch.tensor(labels,dtype = torch.long)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 50 == 0:
            model.eval()
            correct = 0
            total = 0
            for inputs, labels in test_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                .format(epoch+1, num_epochs, i+1, total_step, loss.item()),
                  'Validation Accuracy: {} %'.format(100 * correct / total))



/tmp/ipykernel_817/3075316110.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels,dtype = torch.long)
/tmp/ipykernel_817/2053517580.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc3(x))


Epoch [1/50], Step [50/113], Loss: 2.2434 Validation Accuracy: 21.5 %
Epoch [1/50], Step [100/113], Loss: 2.2225 Validation Accuracy: 21.555555555555557 %
Epoch [2/50], Step [50/113], Loss: 2.1610 Validation Accuracy: 21.38888888888889 %
Epoch [2/50], Step [100/113], Loss: 2.1951 Validation Accuracy: 20.555555555555557 %
Epoch [3/50], Step [50/113], Loss: 2.2080 Validation Accuracy: 21.333333333333332 %
Epoch [3/50], Step [100/113], Loss: 2.2036 Validation Accuracy: 21.0 %
Epoch [4/50], Step [50/113], Loss: 2.2296 Validation Accuracy: 21.666666666666668 %
Epoch [4/50], Step [100/113], Loss: 2.1761 Validation Accuracy: 21.22222222222222 %
Epoch [5/50], Step [50/113], Loss: 2.1840 Validation Accuracy: 20.333333333333332 %
Epoch [5/50], Step [100/113], Loss: 2.2327 Validation Accuracy: 21.38888888888889 %
Epoch [6/50], Step [50/113], Loss: 2.3232 Validation Accuracy: 20.5 %
Epoch [6/50], Step [100/113], Loss: 2.2178 Validation Accuracy: 22.055555555555557 %
Epoch [7/50], Step [50/113], Lo

### Model evalulation

In [8]:
model.eval()
correct = 0
total = 0
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    print(predicted)

print('Accuracy of the network on the 1000 data: {} %'.format(100 * correct / total))



tensor([6, 6, 5, 6, 5, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 5, 6, 6, 5, 6, 6, 5, 6,
        6, 6, 5, 6, 6, 6, 5, 6, 5, 6, 6, 6, 5, 6, 6, 6, 6, 6, 5, 6, 5, 6, 5, 5,
        6, 6, 5, 5, 6, 6, 6, 6, 5, 6, 6, 5, 6, 5, 5, 6], device='cuda:0')
tensor([6, 6, 5, 6, 5, 6, 5, 5, 6, 5, 5, 6, 6, 6, 5, 6, 6, 6, 5, 6, 6, 6, 5, 5,
        5, 6, 5, 6, 6, 6, 6, 5, 5, 5, 6, 6, 6, 5, 6, 6, 6, 5, 6, 6, 6, 5, 6, 6,
        6, 6, 6, 5, 6, 6, 5, 5, 5, 6, 5, 6, 5, 6, 5, 6], device='cuda:0')
tensor([5, 6, 6, 6, 5, 5, 6, 6, 5, 5, 5, 6, 5, 6, 5, 6, 5, 6, 5, 6, 6, 5, 6, 6,
        6, 6, 5, 6, 6, 5, 5, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5,
        6, 6, 6, 6, 6, 5, 6, 5, 6, 6, 5, 5, 6, 6, 5, 6], device='cuda:0')
tensor([6, 6, 6, 6, 5, 6, 6, 6, 6, 5, 5, 6, 6, 5, 6, 5, 6, 6, 5, 5, 6, 6, 5, 5,
        6, 5, 5, 6, 6, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 5, 6, 5, 6, 5, 6,
        5, 6, 6, 6, 5, 6, 6, 6, 6, 6, 5, 5, 6, 6, 6, 5], device='cuda:0')
tensor([6, 5, 6, 6, 6, 6, 6, 5, 5, 6, 6, 6, 6, 6, 5, 5, 5, 6, 6,

/tmp/ipykernel_817/2053517580.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc3(x))


### Save the model

In [9]:
torch.save(model,'save.pt')

In [10]:
torch.load('save.pt')

NN(
  (fc1): Linear(in_features=10, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=10, bias=True)
)

--END--